In [7]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import sys
if not '..' in sys.path:
    sys.path.append('..')
from Pipeline.PPP import *

In [8]:
PATH_PIPE = '../Pipeline/'
PATH_RAW = '../data/raw_data/'
PATH_DATA = '../data/data/'
PATH_MODELS = '../../models/'
PATH_IMAGES = '../assets/images/'

## Load data

Either with separate train and test files or...

In [17]:
train = pd.read_csv(PATH_DATA + "2829_train.csv")
test = pd.read_csv(PATH_DATA + "2829_test.csv")

if 'Unnamed: 0' in train.columns:
    train = train.drop('Unnamed: 0', axis=1)
if 'Unnamed: 0' in test.columns:
    test = test.drop('Unnamed: 0', axis=1)

X, y, y_bin = train.drop(['tag', 'bin_tag'], axis=1), train['tag'], train['bin_tag']
X_test, y_test, y_bin_test = test.drop(['tag', 'bin_tag'], axis=1), test['tag'], test['bin_tag']

X_train, X_val, y_train, y_val = train_test_split(X, y_bin, test_size=0.1, random_state=42)

... with only one file.

In [ ]:
data = pd.read_csv(PATH_DATA + "2840_full_data.csv")

if 'Unnamed: 0' in data.columns:
    train = train.drop('Unnamed: 0', axis=1)

# Prevent missing values
data = data.dropna(subset=['tag'])

X, y, y_bin = data.drop(['tag', 'bin_tag'], axis=1), data['tag'], data['bin_tag']

X_train, X_test, y_train, y_test =  [np.asarray(x) for x in train_test_split(X, y_bin, test_size=0.25, random_state=42)]
X_train, X_val, y_train, y_val = [np.asarray(x) for x in train_test_split(X_train, y_train, test_size=0.1, random_state=42)]

In [18]:
X_train.head()

,text_review,text_pos,lemmas,num_tokens
422,canon has kept the price of this little gem re...,NN VBZ VBN DT NN IN DT JJ NN JJ FW VBD PRP IN ...,canon have keep the price of this little gem r...,276
761,there is no static noise compared to delta's a...,EX VBZ DT JJ NN VBN TO NN POS CC DT NN VBZ JJ ...,there be no static noise compare to delta 's a...,90
2076,great deal they were beautiful and classy,JJ NN PRP VBD JJ CC JJ,great deal they be beautiful and classy,7
1902,the game is fun i would like a little more act...,DT NN VBZ NN FW MD VB DT RB RBR NN CC DT IN DT...,the game be fun i would like a little more act...,20
1740,very cute and sturdy vanity to keep all the pl...,RB JJ CC JJ NN TO VB PDT DT NN NN RB CD JJ MD ...,very cute and sturdy vanity to keep all the pl...,23
